In [1]:
#%pip install xlrd openpyxl scikit-learn tqdm torchviz pandas plotly numpy nbformat tqdm

In [2]:
import pandas as pd
import sklearn as sk
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import tqdm as tqdm
import plotly.express as px

np.version.full_version

'2.1.3'

In [3]:
hope_production_df = pd.read_excel('data.xlsx', sheet_name="HOPE PRODUCTION")
hope_storage_df = pd.read_excel('data.xlsx', sheet_name="HOPE STORAGE AFTER COOKING")

faith_production_df = pd.read_excel('data.xlsx', sheet_name="FAITH PRODUCTION")
faith_storage_df = pd.read_excel('data.xlsx', sheet_name="FAITH STORAGE AFTER COOKING")

hope_faith_df = pd.read_excel('data.xlsx', sheet_name="HOPE-FAITH PACKAGE WEIGHTS")
hope_faith_df.rename(columns={
    x: f"Sample {i + 1}" for i, x in 
    enumerate((x for x in hope_faith_df.columns if "Unnamed" in x))}, inplace=True)

hope_faith_df.columns

Index(['PRODUCT', 'DATE OF PACKING', 'EXPIRY DATE', 'AVERAGE WEIGHT (g)',
       'Sample 1', 'Sample 2', 'Sample 3', 'Sample 4', 'Sample 5', 'Sample 6',
       'Sample 7', 'Sample 8', 'Sample 9', 'Sample 10', 'Sample 11',
       'Sample 12', 'Sample 13', 'Sample 14', 'Sample 15', 'Sample 16',
       'Sample 17', 'Sample 18', 'Sample 19', 'Sample 20', 'Sample 21',
       'Sample 22', 'Sample 23', 'Sample 24', 'Sample 25', 'Sample 26',
       'Sample 27', 'Sample 28', 'Sample 29', 'Sample 30', 'Sample 31',
       'Sample 32'],
      dtype='object')

In [30]:
hope_df = hope_storage_df.merge(hope_production_df, right_on=["BATCH no.", "PRODUCTION DATE"], left_on=["BATCH no.", "BATCH INTO STORAGE"])
faith_df = faith_storage_df.merge(faith_production_df, right_on=["BATCH no.", "PRODUCTION DATE"], left_on=["BATCH no.", "BATCH INTO STORAGE"])

hope_df["PRODUCT"] = 5409
faith_df["PRODUCT"] = 5030

def fix_columns(data: pd.DataFrame, a: str, b: str):
    aa, bb = list(data[a]), list(data[b])
    for i, (xx, yy) in enumerate(zip(aa, bb)):
        aa[i] = max(yy, xx)
        bb[i] = min(yy, xx)
    data[a] = aa
    data[b] = bb
        
def stddev_filter(data: pd.DataFrame, prod: str, col: str):
    for product in data[prod].unique():
        column = data[col]
        filterrr = data[prod] == product
        mean = np.nanmean(column[filterrr])
        std3 = 2 * np.nanstd(column[filterrr])
        data.loc[filterrr, col] = data.loc[filterrr, col].map(lambda v: max(mean - std3, min(v, mean + std3)))

fix_columns(hope_df, "BATCH OUT OF STORAGE", "BATCH INTO STORAGE")
fix_columns(faith_df, "BATCH OUT OF STORAGE", "BATCH INTO STORAGE")
fix_columns(hope_df, "BATCH WEIGHT (kg) BEFORE COOKING", "BATCH WEIGHT (kg) AFTER COOKING")
fix_columns(faith_df, "BATCH WEIGHT (kg) BEFORE COOKING", "BATCH WEIGHT (kg) AFTER COOKING")



HOPE_SHELF_LIFE = 28
FAITH_SHELF_LIFE = 30

hope_df["ESTIMATED EXPIRY"] = hope_df["BATCH INTO STORAGE"] + pd.Timedelta(days=HOPE_SHELF_LIFE)
faith_df["ESTIMATED EXPIRY"] = faith_df["BATCH INTO STORAGE"] + pd.Timedelta(days=FAITH_SHELF_LIFE)

hope_pre_df = hope_df.merge(
        hope_faith_df, left_on=["ESTIMATED EXPIRY", "PRODUCT"], right_on=["EXPIRY DATE", "PRODUCT"]
    )
faith_pre_df = faith_df.merge(
        hope_faith_df, left_on=["ESTIMATED EXPIRY", "PRODUCT"], right_on=["EXPIRY DATE", "PRODUCT"]
    )


pre_df = pd.concat([hope_pre_df, faith_pre_df])
display(pre_df.columns)
pre_df["PRODUCT AMOUNT EXPECTED"] = [1000 * row["BATCH WEIGHT (kg) AFTER COOKING"] / row["AVERAGE WEIGHT (g)"] for _, row in pre_df.iterrows()]


ns = {}
for i, row in pre_df.iterrows():
    prod = row["PRODUCT"]
    if prod not in ns:
        ns[prod] = []
    
    for col in pre_df.columns:
        if "Sample" in col:
            if not pd.isnull(v := row[col]):
                ns[prod].append(v)

for k in ns:
    ns[k] = np.nanstd(ns[k]), np.nanmean(ns[k])

stddev_filter(pre_df, "PRODUCT","AVERAGE WEIGHT (g)")
stddev_filter(pre_df, "PRODUCT", "BATCH WEIGHT (kg) AFTER COOKING")
stddev_filter(pre_df, "PRODUCT", "BATCH WEIGHT (kg) BEFORE COOKING")

pre_df.sort_values('EXPIRY DATE')

Index(['BATCH no.', 'BATCH INTO STORAGE', 'BATCH OUT OF STORAGE',
       'BATCH WEIGHT LEAVING STORAGE (KG)', 'PRODUCTION DATE',
       'BATCH WEIGHT (kg) BEFORE COOKING', 'BATCH WEIGHT (kg) AFTER COOKING',
       'NOTES', 'PRODUCT', 'ESTIMATED EXPIRY', 'DATE OF PACKING',
       'EXPIRY DATE', 'AVERAGE WEIGHT (g)', 'Sample 1', 'Sample 2', 'Sample 3',
       'Sample 4', 'Sample 5', 'Sample 6', 'Sample 7', 'Sample 8', 'Sample 9',
       'Sample 10', 'Sample 11', 'Sample 12', 'Sample 13', 'Sample 14',
       'Sample 15', 'Sample 16', 'Sample 17', 'Sample 18', 'Sample 19',
       'Sample 20', 'Sample 21', 'Sample 22', 'Sample 23', 'Sample 24',
       'Sample 25', 'Sample 26', 'Sample 27', 'Sample 28', 'Sample 29',
       'Sample 30', 'Sample 31', 'Sample 32'],
      dtype='object')

,BATCH no.,BATCH INTO STORAGE,BATCH OUT OF STORAGE,BATCH WEIGHT LEAVING STORAGE (KG),PRODUCTION DATE,BATCH WEIGHT (kg) BEFORE COOKING,BATCH WEIGHT (kg) AFTER COOKING,NOTES,PRODUCT,ESTIMATED EXPIRY,...,Sample 24,Sample 25,Sample 26,Sample 27,Sample 28,Sample 29,Sample 30,Sample 31,Sample 32,PRODUCT AMOUNT EXPECTED
0,1637,2023-10-09,2023-10-09,286.0,2023-10-09,290.833728,286.000000,NaN,5030,2023-11-08,...,586,584,582,581,585,585,583,587.0,582.0,489.647424
209,1823,2023-10-09,2023-10-10,262.6,2023-10-09,265.800000,262.600000,NaN,5030,2023-11-08,...,582,583,585,585,583,584,587,581.0,582.0,450.042845
208,1823,2023-10-09,2023-10-10,262.6,2023-10-09,265.800000,262.600000,NaN,5030,2023-11-08,...,585,590,590,592,595,596,582,586.0,591.0,445.769455
207,1823,2023-10-09,2023-10-10,262.6,2023-10-09,265.800000,262.600000,NaN,5030,2023-11-08,...,586,584,582,581,585,585,583,587.0,582.0,449.585362
206,1822,2023-10-09,2023-10-10,268.7,2023-10-09,273.600000,268.700000,NaN,5030,2023-11-08,...,582,583,585,585,583,584,587,581.0,582.0,460.497001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30686,4054,2024-10-04,2024-10-07,270.1,2024-10-04,275.600000,270.100000,NaN,5030,2024-11-03,...,580,564,574,575,579,581,570,578.0,588.0,465.714748
30687,4054,2024-10-04,2024-10-07,270.1,2024-10-04,275.600000,270.100000,NaN,5030,2024-11-03,...,585,585,595,589,591,588,588,584.0,587.0,459.695777
30688,4055,2024-10-04,2024-10-07,258.5,2024-10-04,263.600000,258.500000,NaN,5030,2024-11-03,...,580,564,574,575,579,581,570,578.0,588.0,445.713670
30678,4049,2024-10-04,2024-10-07,274.1,2024-10-04,279.000000,274.100000,NaN,5030,2024-11-03,...,580,564,574,575,579,581,570,578.0,588.0,472.611671


In [31]:
from sklearn import preprocessing

df = pd.DataFrame()

df['product'] = pre_df['PRODUCT']

df['time_in_storage'] = [x / np.timedelta64(1, 'D') for x in (pre_df["BATCH OUT OF STORAGE"] - pre_df["BATCH INTO STORAGE"]).values]
df['weight_difference'] = pre_df["BATCH WEIGHT (kg) BEFORE COOKING"] - pre_df["BATCH WEIGHT (kg) AFTER COOKING"]
stddev_filter(df, "product","weight_difference")
df['weekday'] = [x.weekday() for x in pre_df["BATCH INTO STORAGE"]]
df['weekday'] = df['weekday'] / max(df['weekday'])
df['input_amount'] = preprocessing.MinMaxScaler().fit_transform(pre_df["BATCH WEIGHT (kg) BEFORE COOKING"].values.reshape(-1, 1))
df['cooking_out__storage_in'] = preprocessing.MinMaxScaler().fit_transform(pre_df["BATCH WEIGHT (kg) AFTER COOKING"].values.reshape(-1, 1))
df['storage_out__packaging_in'] = preprocessing.MinMaxScaler().fit_transform(pre_df["BATCH WEIGHT LEAVING STORAGE (KG)"].values.reshape(-1, 1))
df['product_amount_expected'] = preprocessing.MinMaxScaler().fit_transform(pre_df["PRODUCT AMOUNT EXPECTED"].values.reshape(-1, 1))
df['mean'] = preprocessing.MinMaxScaler().fit_transform(pre_df["AVERAGE WEIGHT (g)"].values.reshape(-1, 1))
df['stdev'] = df["product"].map(lambda x: ns[x][0])

df.sort_values('weight_difference')

,product,time_in_storage,weight_difference,weekday,input_amount,cooking_out__storage_in,storage_out__packaging_in,product_amount_expected,mean,stdev
30817,5030,3.0,1.989398,0.666667,0.665344,0.707905,0.807899,0.690750,0.988682,8.342451
30816,5030,3.0,1.989398,0.666667,0.665344,0.707905,0.807899,0.699840,0.947097,8.342451
30815,5030,3.0,1.989398,0.666667,0.694199,0.722811,0.812954,0.695072,0.988682,8.342451
30814,5030,3.0,1.989398,0.666667,0.694199,0.722811,0.812954,0.704219,0.947097,8.342451
30801,5030,3.0,1.989398,0.666667,0.753899,0.779642,0.832227,0.711551,0.988682,8.342451
...,...,...,...,...,...,...,...,...,...,...
727,5409,1.0,12.357578,0.500000,0.168839,0.118166,0.607899,0.754979,0.000000,5.318712
728,5409,1.0,12.357578,0.500000,0.184759,0.139594,0.615166,0.763986,0.000000,5.318712
729,5409,1.0,12.357578,0.500000,0.220579,0.158227,0.621485,0.771817,0.000000,5.318712
730,5409,1.0,12.357578,0.500000,0.220579,0.160090,0.622117,0.772601,0.000000,5.318712


In [32]:
#px.scatter(df, x="time_in_storage", y="storage_out__packaging_in")
px.scatter(df, x="weight_difference", y="cooking_out__storage_in")

In [7]:
class Data(torch.utils.data.Dataset):
    def __init__(self):
        self.data = df
        self.target = ["input_amount","cooking_out__storage_in", "storage_out__packaging_in"] 
    
    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index) -> tuple[torch.Tensor, torch.Tensor]:
        row = self.data.iloc[index]
        x = []
        y = []
        for k, v in row.items():
            if k in self.target:
                y.append(v)
            else:
                x.append(v)
        return tuple(map(torch.Tensor, [x, y]))

data = Data()
data[0]

(tensor([5.4090e+03, 0.0000e+00, 5.0000e-01, 7.5276e-01, 3.1817e-02, 5.3187e+00]),
 tensor([0.6539, 0.6101, 0.6101]))

In [8]:
INPUT_SIZE = len(data[0][0])
OUTPUT_SIZE = len(data[0][1])
INTERNAL = 12

class Net(nn.Module):
    def __init__(self):
            super(Net, self).__init__()
            self.l1 = nn.Linear(INPUT_SIZE, INTERNAL)
            self.lrelu = nn.LeakyReLU()
            self.l2 = nn.Linear(INTERNAL, INTERNAL)
            self.relu = nn.ReLU()
            self.l3 = nn.Linear(INTERNAL, INTERNAL)
            self.silu = nn.SiLU()
            self.l4 = nn.Linear(INTERNAL, OUTPUT_SIZE)
    
    def forward(self, x):
        return self.l4(self.silu(self.l3(self.relu(self.l2((self.lrelu(self.l1(x))))))))

INPUT_SIZE, INTERNAL, INTERNAL, OUTPUT_SIZE

(6, 12, 12, 3)